In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torchvision.transforms import ToPILImage
from PIL import Image

import numpy as np 
import pandas as pd
import os
import time
import random

show = ToPILImage()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
path = 'E:\InteliJ Programes\Pycharm\dataset\kaggle\kaggleFashion'

train_path = os.path.join(path, 'fashion-mnist_train.csv')
test_path = os.path.join(path, 'fashion-mnist_test.csv')

train_csv = pd.read_csv(train_path)
test_csv = pd.read_csv(test_path)

In [3]:
train_csv.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
class FashionMNIST(Dataset):
    def __init__(self, data, transform=None):
        self.transform = transform
        self.labels = data.label.values
        self.images = data.iloc[:, 1:].values.astype('uint8').reshape(-1, 28, 28)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        label = self.labels[index]
        image = Image.fromarray(self.images[index])
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [5]:
transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])

train_set = FashionMNIST(train_csv, transform)
test_set = FashionMNIST(test_csv, transform)

In [6]:
batch_size = 64
train_loader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    )

test_loader = DataLoader(
    test_set,
    batch_size=batch_size,
    )

In [7]:
classes = ["T-shirt/Top",
         "Trouser",
         "Pullover",
         "Dress",
         "Coat", 
         "Sandal", 
         "Shirt",
         "Sneaker",
         "Bag",
         "Ankle Boot"]

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(   #[-1, 28, 28]
            nn.Conv2d(1, 32, 3, padding=1), #[32, 28, 28]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2), #[32, 14, 14]
            nn.Conv2d(32, 64, 3), #[64, 12, 12]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2) #[64, 6, 6]  
        )
        self.dense = nn.Sequential(
            nn.Linear(64*6*6, 600), 
            nn.Dropout(0.25),
            nn.Linear(600, 120),
            nn.Linear(120, 10)            
        )
    def forward(self,x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        x = self.dense(x)
        return x

In [9]:
model = Net()
model.to(device)

Net(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=2304, out_features=600, bias=True)
    (1): Dropout(p=0.25, inplace=False)
    (2): Linear(in_features=600, out_features=120, bias=True)
    (3): Linear(in_features=120, out_features=10, bias=True)
  )
)

In [10]:
learning_rate = 1e-3

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    epoch_start = time.time()
    train_acc =0.0
    train_loss = 0.0
    print("------------第{}轮训练开始-----------".format(epoch+1))
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        images, labels = data[0].to(device), data[1].to(device)
        output = model(images)
        
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()
        
        train_acc += (torch.argmax(output, axis=1) == labels).sum()
        train_loss += loss.item()
    
    print("[{}/{}]，{:.2f} sec(s), Train_acc:{:7f}，Train_loss:{:7f}，".format(epoch+1, num_epochs,time.time()-epoch_start, train_acc/train_set.__len__(), train_loss/train_set.__len__()))        

------------第1轮训练开始-----------
[1/5]，19.12 sec(s), Train_acc:0.849333，Train_loss:0.006642，
------------第2轮训练开始-----------
[2/5]，16.28 sec(s), Train_acc:0.892233，Train_loss:0.004702，
------------第3轮训练开始-----------
[3/5]，16.25 sec(s), Train_acc:0.905883，Train_loss:0.004057，
------------第4轮训练开始-----------


In [ ]:
torch.save(model, 'E:\InteliJ Programes\Pycharm\Pytorch\Kaggle\models\FashionMNIST{}.pth'.format(epochs))

In [ ]:
model.eval()
with torch.no_grad():
    test_acc = 0.
    test_loss = 0.
    epoch_start = time.time()
    
    for i, data in enumerate(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        output = model(images)
        
        loss = criterion(output, labels.long())
        
        test_acc += (torch.argmax(output, axis=1) == labels).sum()
        test_loss += loss.item()
    
    print("{:.2f} sec(s), Test_acc:{:5f}，Test_loss:{:5f}，".format(time.time()-epoch_start, test_acc/test_set.__len__(), test_loss/test_set.__len__()))      
    

In [ ]:
def random_test(num):
    test_image, test_label = test_set[num]
    print(classes[test_label])
    test_image = test_image.unsqueeze(0).to(device)
    out = model(test_image)
    result = result = torch.argmax(out, axis=1)
    return classes[result]

In [ ]:
stochastic = random.randint(0, len(test_set))
random_test(stochastic)